## Импорт нужных модулей

In [ ]:
import requests
from datetime import datetime
import sqlite3
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from pymystem3 import Mystem
m = Mystem()
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
from tqdm.auto import tqdm
import re

## Нужные данные
- ``token`` — токен приложения;

In [ ]:
token = '???' #Сюда вставить токен
version = '5.92'

## База данных
- подключение;
- создание нужных таблиц

In [ ]:
conn = sqlite3.connect('my_groups.db')
cur = conn.cursor()

In [ ]:
cur.execute("""
CREATE TABLE IF NOT EXISTS posts 
(post_id text PRIMARY KEY, post_text text, post_lemmas text, smiles text, pos text,
date text, likes int, hashtag text, domain text)
""")
conn.commit()

## Функция для лемматизирования и очистки тектса
Ещё она делает список использованных смайликов, а сами смайлики выкидывает из текста, но почему-то работает не для всех символов, нужно смотреть таблицы дальше.

In [ ]:
def lemmas(my_text):
    emojis = []
    #pos = []
    my_pos = ''
    tag = ''
    if '#' in my_text:
        new_text = my_text.split('#')[0]
        tag = my_text.split('#')[1]
        if '@' in tag:
            tag = tag.split('@')[0]
    else:
        new_text = my_text
    tokens = word_tokenize(new_text)
    tokens = [i for i in tokens if (i not in string.punctuation)]
    stop_words = stopwords.words('russian')
    tokens = [i for i in tokens if (i not in stop_words)]
    two_text = ' '.join(tokens)
    two_text = two_text.replace('.', '').replace('«', '').replace('»', '')
    lemmas = m.lemmatize(two_text)
    lemma_text = ''.join(lemmas[:-1])
    for l in lemmas:
            emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"
            u"\U0001F300-\U0001F5FF"
            u"\U0001F680-\U0001F6FF"
            u"\U0001F1E0-\U0001F1FF"
            u"\U0001F6D0-\U0001FAD6"
            u"\U00012600-\U000126FF"
                               "]+", flags=re.UNICODE)
            emo = re.match(emoji_pattern, l)
            if not emo == None:
                emojis.append(emo.group())
                lemma_text = lemma_text.replace(emo.group(), '')
            ana = morph.parse(l)
            pos = ana[0].tag.POS
            if not pos == None:
                if str(pos) == 'INFN':
                    pos = 'VERB'
                my_pos += str(pos) + ' '
    new_text = re.sub(r'  +', ' ', new_text)
    my_emojis = ''.join(emojis)
    return new_text, lemma_text, my_emojis, my_pos, tag

## Функция, которая получает 2000 постов, и для каждого из них:
- находит текст поста;
- находит лемматизированный текст;
- находит дату написания;
- получает число лайков на посте;
- записывает это всё в базу, если текст — не пустой.

Возвращает текст и число записанных постов.

При этом удовлетворительными признаются посты, в которых 40 или более слов.

In [ ]:
def find_posts(my_domain, offset_list):
    my_conn = sqlite3.connect('my_groups.db')
    cur = my_conn.cursor()
    one_count = 0
    text = None
    for o in offset_list:
        my_data = requests.get(
        'https://api.vk.com/method/wall.get',
        params={
            'domain': my_domain,
            'access_token': token,
            'v': version,
            'count': 100,
            'offset': o
            }).json()
        #print(my_data)
        if 'response' in my_data.keys():
            for i in my_data['response']['items']:
                if len(i['text'].split(' ')) >= 40:
                    one_count += 1
                    text = i['text']
                    my_new_text, lemmatized, emoj, pos, tags = lemmas(text)
                    my_post_id = str(i['owner_id'])+'_'+str(i['id'])
                    date = i['date']
                    my_date = datetime.fromtimestamp(date)
                    my_likes  = i['likes']['count']              
                    my_conn = sqlite3.connect('my_groups.db')
                    cur = my_conn.cursor()

                    sqlite_insert_with_param = """INSERT INTO posts
                                      (post_id, post_text, post_lemmas, smiles, pos, date, likes, hashtag, domain)
                                      VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?);"""

                    data_tuple = (my_post_id, my_new_text, lemmatized, emoj, pos, my_date, my_likes, tags, my_domain)
                    #print(data_tuple)
                    try:
                        cur.execute(sqlite_insert_with_param, data_tuple)
                        my_conn.commit()
                    except Exception:
                        print('Это что ещё такое?')
            if one_count >= 2000:
                break
        print(one_count)
    return(text, one_count)

**Список групп** и их короткие адреса:

- **Подслушано** overhear **+**
- **Just Story** just_str **+**
- **My story** my_storry **+**

- **Палата**:
   - **Палата №6** pn6 **+-**
   - **Мамдаринка** momdarinka +
   - **Шкогвартс** shkogwarts +
   - **Палата №666** palnom_666 +
   - **Палата №69** palatanom69 +
   - **Карамель** caramel6 +
   - **Не все поймут** nevsep
   
- **Страшные истории** horror_tales

In [ ]:
groups = [] #Сюда надо добавить названия групп.
#Я делала это не сразу, потому что иначе требовалось слишком много времени.
for group in groups:
    find_posts(group, [0, 100, 200, 300, 400, 500,
                      600, 700, 800, 900, 1000,
                      1100, 1200, 1300, 1400, 1500,
                      1600, 1700, 1800, 1900, 2000,
                      2100, 2200, 2300, 2400, 2500,
                      2600, 2700, 2800, 2900, 3000,
                      3100, 3200, 3300, 3400, 3500, 3600, 3700, 3800, 3900, 4000,
                      4100, 4200, 4300, 4400, 4500, 4600, 4700, 4800, 4900, 5000,
                      5100, 5200, 5300, 5400, 5500, 5600, 5700, 5800, 5900, 6000])